In [4]:
# Azure 서버에서 패키지를 설치 (필요한 것  CustomVision)
%pip install azure-cognitiveservices-vision-customvision

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 설치한 Azure Pakage에서 기능 import

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from msrest.authentication import ApiKeyCredentials
#Credentials : key보다 더 높은 보안성을 가지고 권한을 부여함. -> credentials를 사용할 수 있는 package

In [2]:
# Azure Resource의 미리 만들어둔 endpoint와 key값 입력

# training 객체 생성성
endpoint = 'https://labuser10customvision.cognitiveservices.azure.com/'
training_key = 'ed10c92c84654102881c541d801308e2'

In [7]:
# publish 할 모델의 이름을 지정

publish_interation_name = 'GreatWall_Model_10'
credentials = ApiKeyCredentials(in_headers={'Training-key': training_key})

In [4]:
# train 객체를 만드는 작업

trainer = CustomVisionTrainingClient(endpoint, credentials)

In [ ]:
# Create a new project / 새로운 프로젝트 작업

print('Creating project....')
project_name = 'GreatWall_10'   # 프로젝트 이름 지정
project = trainer.create_project(project_name)  # project_name(GreatWall_10)이라는 이름으로 프로젝트 메모리에 생성

# www.customvision.ai에 프로젝트가 생성됨!

In [ ]:
# 프로젝트 내에 Tag 생성
# customvision.ai에서 이미지 업로드

JjaJangMien_tag = trainer.create_tag(project.id, 'JjaJangMien')
JjangPon_tag = trainer.create_tag(project.id, 'JjangPon')
BangSuYuk_tag = trainer.create_tag(project.id, 'BangSuYuk')

In [ ]:
# 학습의 시작
import time # time 이라는 module을 사용해서 10초 대기 시간 부여

print('Training......')
iteration = trainer.train_project(project.id) # project.id 에서 학습을 시작하겠다. -> iteration 이라는 이름으로 저장
while (iteration.status != 'Completed'):      # 학습 상태가 아니면 while에서 끊겠다.completed 상태로 종료.
  iteration = trainer.get_iteration(project.id, iteration.id)
  print('Training status: ' + iteration.status) # 학습 상태일 때
  print('Waiting 10 sceonds...')  # 10초에 한 번씩 찍어야함
  time.sleep(10)    # 10초에 한 번 씩 프로젝트가 끝났는지 체크 -> 10초 마다 상태가 Completed인지 확인 while training end

In [ ]:
# 예측, 추론하는 부분

# predict clinet를 가져와서 예측을 할거다.
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

In [ ]:
# predict client의 key와 endpoint를 입력력

prediction_key = 'faf623cd34134901b2d7bbdcc29fc9c4'
prediction_endpoint = 'https://labuser10customvision-prediction.cognitiveservices.azure.com/'

prediction_credential = ApiKeyCredentials(
    in_headers={'Prediction-key': prediction_key})

In [ ]:
# 예측하는 리소스의 생성

predictor = CustomVisionPredictionClient(prediction_endpoint, 
                                         prediction_credential) # (주소,credential)을 입력해서 클라이언트에 접속

In [ ]:
# 예측할 이미지의 주소를 입력 (구글링 : 짜장면)

target_image_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSO-lQuygALNs_owASbp9RL4rTnMzwz0zM2pw&usqp=CAU'
result = predictor.classify_image_url(project.id,
                                      'Iteration1',   # customvision.ai에서 publish 한 이름을 가져와서
                                      target_image_url)    # img의 url을 가지고 실제로 predictor가 분석을 실행하도록 함

In [ ]:
# 예측한 결과들 불러오기

for prediction in result.predictions :   # 예측한 결과들이 반복적으로 나타나게됨
  print('\t'+ prediction.tag_name + ': {0:.2f}%'.format(prediction.probability * 100) ) 
# 결과값의 포맷을 정해줌 -> 소수점 둘째자리 까지 0:2f
# probability : 1을 기준으로 퍼센테이지로 보이기 위해서 100을 곱함 -> predictiom 객체의 속성이라 괄호가 없음, 메소드
# 속성은 변수 / 메소드는 함수

# 각각의 태그 네임에 따라서 몇 플